In [6]:
import pandas as pd 
import numpy as np 
import random 
from sklearn.impute import KNNImputer, SimpleImputer
import sklearn
import matplotlib.pyplot as plt

df = pd.read_csv('./outliers.csv')
df

,DATE_TODAY,ID_ARTICLE,LIBELLE,PRICE,STOCK
0,2020-09-07,2196543,NaN,1.49,18.0
1,2020-09-07,2339672,NaN,1.49,47.0
2,2020-09-07,2196543,Yaourt 0% myrtille/framboise,1.49,18.0
3,2020-09-07,2024529,NaN,2.69,19.0
4,2020-09-07,8718949,NaN,1.89,27.0
...,...,...,...,...,...
751,2020-09-07,1866590,NaN,2.30,633.0
752,2020-09-07,2390619,"Yaourt, panaché",4.99,11.0
753,2020-09-07,1866590,NaN,2.30,645.0
754,2020-09-07,1866565,Yaourts au céréales,2.30,24.0


In [32]:
# Sélectionner uniquement les colonnes numériques
df_numeric = df.select_dtypes(include=[np.number])

# Fonction pour détecter les valeurs aberrantes en utilisant la méthode IQR
def detect_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((df < lower_bound) | (df > upper_bound))
    return outliers

# Détection des valeurs aberrantes dans les colonnes numériques
outliers_iqr = detect_outliers_iqr(df_numeric)
outliers_iqr


,ID_ARTICLE,PRICE,STOCK
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
751,False,False,True
752,False,True,False
753,False,False,True
754,False,False,False


In [34]:
# Compter les valeurs aberrantes par colonne et au total
print(outliers_iqr.sum())
print(outliers_iqr.sum().sum())

ID_ARTICLE      0
PRICE         101
STOCK         127
dtype: int64
228


In [38]:
df_cleaned_iqr = df[~outliers_iqr.any(axis=1)]

# Affichage du DataFrame après suppression des valeurs aberrantes
print("\nDataFrame après suppression des valeurs aberrantes :")
print(df_cleaned_iqr)


DataFrame après suppression des valeurs aberrantes :
     DATE_TODAY  ID_ARTICLE                                   LIBELLE  PRICE  \
0    2020-09-07     2196543                                       NaN   1.49   
1    2020-09-07     2339672                                       NaN   1.49   
2    2020-09-07     2196543              Yaourt 0% myrtille/framboise   1.49   
3    2020-09-07     2024529                                       NaN   2.69   
4    2020-09-07     8718949                                       NaN   1.89   
..          ...         ...                                       ...    ...   
746  2020-09-07     1718387     Yaourt à la grecque mûre et framboise   1.99   
748  2020-09-07     1866590              Yaourts au céréales – muesli   2.25   
750  2020-09-07     2133325  Yaourt à la grecque, cheesecake myrtille   3.49   
754  2020-09-07     1866565                       Yaourts au céréales   2.30   
755  2020-09-07     9768180                        Yaourt – frambo

In [42]:
from scipy.stats import zscore

df_numeric_zscore = df.select_dtypes(include=[np.number])

# Calcul du Z-Score pour chaque valeur dans les colonnes numériques
z_scores = df_numeric_zscore.apply(zscore)

# Détection des valeurs aberrantes en utilisant le seuil de Z-Score > 3 ou < -3
outliers_zscore = (z_scores.abs() > 3)
outliers_zscore

,ID_ARTICLE,PRICE,STOCK
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
751,False,False,True
752,False,False,False
753,False,False,True
754,False,False,False


In [44]:
print(outliers_zscore.sum())
print(outliers_zscore.sum().sum())

ID_ARTICLE     0
PRICE         18
STOCK         20
dtype: int64
38


In [46]:
df_cleaned_zscore = df[~outliers_zscore.any(axis=1)]
df_cleaned_zscore

,DATE_TODAY,ID_ARTICLE,LIBELLE,PRICE,STOCK
0,2020-09-07,2196543,NaN,1.49,18.0
1,2020-09-07,2339672,NaN,1.49,47.0
2,2020-09-07,2196543,Yaourt 0% myrtille/framboise,1.49,18.0
3,2020-09-07,2024529,NaN,2.69,19.0
4,2020-09-07,8718949,NaN,1.89,27.0
...,...,...,...,...,...
749,2020-09-07,6699475,NaN,1.20,84.0
750,2020-09-07,2133325,"Yaourt à la grecque, cheesecake myrtille",3.49,19.0
752,2020-09-07,2390619,"Yaourt, panaché",4.99,11.0
754,2020-09-07,1866565,Yaourts au céréales,2.30,24.0


Comparer les deux méthodes 

In [50]:
# Comparaison des DataFrames nettoyés
print("\nLignes supprimées par IQR mais pas par Z-Score :")
print(df[~df.index.isin(df_cleaned_iqr.index) & df.index.isin(df_cleaned_zscore.index)])

print("\nLignes supprimées par Z-Score mais pas par IQR :")
print(df[~df.index.isin(df_cleaned_zscore.index) & df.index.isin(df_cleaned_iqr.index)])

print("\nValeurs aberrantes détectées avec Z-Score :")
print(outliers_zscore)
print("\nValeurs aberrantes détectées par colonne (Z-Score) :")
print(outliers_zscore.sum())
print("Nombre total de valeurs aberrantes détectées (Z-Score) :", outliers_zscore.sum().sum())

print("Valeurs aberrantes détectées avec IQR :")
print(outliers_iqr)
print("\nValeurs aberrantes détectées par colonne (IQR) :")
print(outliers_iqr.sum())
print("Nombre total de valeurs aberrantes détectées (IQR) :", outliers_iqr.sum().sum())



Lignes supprimées par IQR mais pas par Z-Score :
     DATE_TODAY  ID_ARTICLE               LIBELLE  PRICE  STOCK
7    2020-09-07     9768180                   NaN   2.69  109.0
8    2020-09-07     2390635  Yaourt 0%, framboise   2.29  250.0
16   2020-09-07     2209445                   NaN   1.49  206.0
32   2020-09-07     1522793                   NaN   9.46   22.0
36   2020-09-07     2051955                   NaN   4.99   24.0
..          ...         ...                   ...    ...    ...
741  2020-09-07     1545914                   NaN   2.73  152.0
744  2020-09-07     2196534      Yaourt 0% fraise   5.70   24.0
747  2020-09-07     9346084                   NaN   3.55  152.0
749  2020-09-07     6699475                   NaN   1.20   84.0
752  2020-09-07     2390619       Yaourt, panaché   4.99   11.0

[178 rows x 5 columns]

Lignes supprimées par Z-Score mais pas par IQR :
Empty DataFrame
Columns: [DATE_TODAY, ID_ARTICLE, LIBELLE, PRICE, STOCK]
Index: []

Valeurs aberrantes détec

Detecter, compter et supprimer les valeurs aberrantes à l'aide de l'isolation forest 

In [58]:
from sklearn.ensemble import IsolationForest 
# Sélectionner uniquement les colonnes numériques
df_numeric_IF = df.select_dtypes(include=[np.number])

# Initialiser l'Isolation Forest
iso_forest = IsolationForest(contamination=0.2)  # contamination est une estimation du pourcentage d'anomalies

# Appliquer l'Isolation Forest
df_numeric_IF['anomaly'] = iso_forest.fit_predict(df_numeric)

# Détecter les valeurs aberrantes (anomalies sont marquées par -1)
outliers_if = df_numeric_IF['anomaly'] == -1
outliers_if
df_numeric_IF

,ID_ARTICLE,PRICE,STOCK,anomaly
0,2196543,1.49,18.0,1
1,2339672,1.49,47.0,1
2,2196543,1.49,18.0,1
3,2024529,2.69,19.0,1
4,8718949,1.89,27.0,1
...,...,...,...,...
751,1866590,2.30,633.0,-1
752,2390619,4.99,11.0,1
753,1866590,2.30,645.0,-1
754,1866565,2.30,24.0,1


In [60]:
outliers_count = outliers_if.sum()
print("\nNombre total de valeurs aberrantes détectées avec Isolation Forest :", outliers_count)

# Supprimer les lignes contenant des valeurs aberrantes
df_cleaned_if = df[~outliers_if]

# Affichage du DataFrame après suppression des valeurs aberrantes
print("\nDataFrame après suppression des valeurs aberrantes :")
print(df_cleaned_if)


Nombre total de valeurs aberrantes détectées avec Isolation Forest : 151

DataFrame après suppression des valeurs aberrantes :
     DATE_TODAY  ID_ARTICLE                                   LIBELLE  PRICE  \
0    2020-09-07     2196543                                       NaN   1.49   
1    2020-09-07     2339672                                       NaN   1.49   
2    2020-09-07     2196543              Yaourt 0% myrtille/framboise   1.49   
3    2020-09-07     2024529                                       NaN   2.69   
4    2020-09-07     8718949                                       NaN   1.89   
..          ...         ...                                       ...    ...   
749  2020-09-07     6699475                                       NaN   1.20   
750  2020-09-07     2133325  Yaourt à la grecque, cheesecake myrtille   3.49   
752  2020-09-07     2390619                           Yaourt, panaché   4.99   
754  2020-09-07     1866565                       Yaourts au céréales   